## fill_imaging_batch_table
It became clear that imaging batches were going to be necessary, just like clearing batches. The solution to this is to create an ImagingBatch() table in the schema just like ClearingBatch() and then add a foreign key to the Sample() table to ImagingBatch(). In order to do this, the ImagingBatch() table needs to be pre-populated before the foreign key can be added to a revised Sample() table. 

In this notebook, I populate the ImagingBatch() table from existing requests. 

In [1]:
import pickle
import os.path, sys
from datetime import datetime
import pandas as pd
import numpy as np
import datajoint as dj
dj.config["enable_python_native_blobs"] = True # So I can store python dictionaries in blob columns

## Connect to the db

In [2]:
dj.config['database.host'] = 'datajoint00.pni.princeton.edu'
dj.conn()

Please enter DataJoint username: ahoag
Please enter DataJoint password: ········
Connecting ahoag@datajoint00.pni.princeton.edu:3306


DataJoint connection (connected) ahoag@datajoint00.pni.princeton.edu:3306

In [12]:
# set up object for light sheet schema
# db_lightsheet = dj.create_virtual_module('ahoag_lightsheet_demo','ahoag_lightsheet_demo')
db_lightsheet = dj.create_virtual_module('u19lightserv_lightsheet','u19lightserv_lightsheet')

## Setup for the ingestion

In [4]:
# Here are the columns
db_lightsheet.Request.ImagingBatch()

username user in the lab,request_name,imaging_batch_number,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,imaging_dict


Can get most of this information from the ImagingRequest() table.

The thing I need to be careful of is how to assign imaging_batch_number. I need to actually figure out the imaging batches based on what resolution and channels were requested in each request. 

In [5]:
request_contents = db_lightsheet.Request().fetch(as_dict=True)
sample_contents = db_lightsheet.Request.Sample()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
channel_contents = db_lightsheet.Request.ImagingChannel()

In [7]:
imaging_batch_insert_list = []
for request_dict in request_contents:
    imaging_dict = {}
    username = request_dict['username']
    request_name = request_dict['request_name']
    print(f"Username: {username}")
    print(f"Request Name: {request_name}")
    print()
    date_submitted = request_dict['date_submitted']
    time_submitted = request_dict['time_submitted']
    imaging_batch_master_dict = {
        'username':username,
        'request_name':request_name,
        'imaging_request_date_submitted':date_submitted,
        'imaging_request_time_submitted':time_submitted,
        'imaging_progress':'incomplete'
    }
    # loop through samples and figure out imaging batches
    sample_contents_this_request = (sample_contents & \
        {'username':username,'request_name':request_name}).fetch(as_dict=True)
    
    for sample_dict in sample_contents_this_request:
        sample_name = sample_dict['sample_name']
        
        imaging_dict[sample_name] = {}
        channel_contents_this_sample = channel_contents & \
            {'username':username,'request_name':request_name,'sample_name':sample_name}
        for channel_dict in channel_contents_this_sample:
            image_resolution = channel_dict['image_resolution']
            channel_name = channel_dict['channel_name']
            if image_resolution in imaging_dict[sample_name].keys():
                imaging_dict[sample_name][image_resolution].append(channel_name)
            else:
                imaging_dict[sample_name][image_resolution] = [channel_name]
#     print(imaging_dict)
    # Count up numbers in each batch
    used_dicts_dict = {} # {batch_number:imaging_dict}
    batch_counter_dict = {} # {1:{'number_in_batch':1,'imaging_dict':{},}}
    batch_number = 0
    for sample_name in imaging_dict.keys():
        if imaging_dict[sample_name] in used_dicts_dict.values():
#             print(f"sample {sample_name} belongs to an existing imaging batch")
            # Figure out the imaging batch this sample belongs to
            this_batch_number = [key for key in used_dicts_dict if used_dicts_dict[key] == imaging_dict[sample_name]]

            batch_counter_dict[batch_number]['number_in_imaging_batch'] +=1
            continue
        else:
            batch_number +=1
#             print(f"sample {sample_name} is first in batch number {batch_number}")
            # make a new entry
            batch_counter_dict[batch_number]={'number_in_imaging_batch':1,
                'imaging_dict':imaging_dict[sample_name]}
            used_dicts_dict[batch_number] = imaging_dict[sample_name]
            
    # Loop through numbers_in_sample_dict and make batches
    for batch_number in batch_counter_dict.keys():
        imaging_batch_insert_dict = imaging_batch_master_dict.copy()
        imaging_batch_insert_dict['imaging_batch_number'] = batch_number
        number_in_batch = batch_counter_dict[batch_number]['number_in_imaging_batch']
        imaging_batch_insert_dict['number_in_imaging_batch'] = number_in_batch
        imaging_dict_this_batch = batch_counter_dict[batch_number]['imaging_dict']
        imaging_batch_insert_dict['imaging_dict'] = imaging_dict_this_batch
        imaging_batch_insert_list.append(imaging_batch_insert_dict)
# print(imaging_batch_insert_list)
db_lightsheet.Request.ImagingBatch.insert(imaging_batch_insert_list,skip_duplicates=True)
# imaging_dict
        

Username: afalkner
Request Name: MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6

Username: apv2
Request Name: 20190313_IBL_DiI_1

Username: apv2
Request Name: ibl_witten_04

Username: cz15
Request Name: zimmerman_01

Username: eguthman
Request Name: mg80175

Username: ejdennis
Request Name: 10-13_brains,_names_TBD

Username: ejdennis
Request Name: 201905_atlas00x_where_x=1:n

Username: ejdennis
Request Name: 20190606_atlas00x_where_x=11-20

Username: ejdennis
Request Name: E112,_E126,_E137

Username: ejdennis
Request Name: K310_(CM-diI),_K315_(CM-diI),_K320,_K321,_K323,_K327,_K333,_K334

Username: ejdennis
Request Name: three_female_atlas_brains

Username: ejdennis
Request Name: W118,_K292,_K293,_K295,_K301,_K302,_K303,_K304,_K305,_K306,_K307

Username: ejdennis
Request Name: W122,_W128,_E111

Username: ejdennis
Request Name: X013

Username: ejdennis
Request Name: X042

Username: ejdennis
Request Name: X050

Username: gerardjb
Request Name: Aldolase_C_Titration

Username: jverp

In [8]:
# Here are the columns
db_lightsheet.Request.ImagingBatch()

username user in the lab,request_name,imaging_batch_number,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,imaging_dict
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",1,None,8,2019-11-18,17:22:11,None,incomplete,=BLOB=
apv2,20190313_IBL_DiI_1,1,None,1,2019-03-13,1:43:03,None,incomplete,=BLOB=
apv2,ibl_witten_04,1,None,1,2019-08-12,15:07:23,None,incomplete,=BLOB=
cz15,zimmerman_01,1,None,2,2020-10-27,10:10:45,None,incomplete,=BLOB=
eguthman,mg80175,1,None,1,2020-09-29,13:08:31,None,incomplete,=BLOB=
ejdennis,"10-13_brains,_names_TBD",1,None,1,2019-08-06,17:12:15,None,incomplete,=BLOB=
ejdennis,201905_atlas00x_where_x=1:n,1,None,1,2019-05-21,19:12:45,None,incomplete,=BLOB=
ejdennis,20190606_atlas00x_where_x=11-20,1,None,10,2019-06-04,10:43:37,None,incomplete,=BLOB=
ejdennis,"E112,_E126,_E137",1,None,3,2020-03-11,12:03:39,None,incomplete,=BLOB=
ejdennis,"K310_(CM-diI),_K315_(CM-diI),_K320,_K321,_K323,_K327,_K333,_K334",1,None,8,2020-02-16,14:11:46,None,incomplete,=BLOB=


OK, that seemed to work. Now let's make the inserts for the new SampleCopy() table with the correct imaging batch numbers

In [9]:
sample_reinsert_list = []
for sample_dict in sample_contents:
    # Figure out which batch this corresponds to 
    sample_reinsert_dict = sample_dict.copy()
    username = sample_dict['username']
    sample_name = sample_dict['sample_name']
    request_name = sample_dict['request_name']
    imaging_dict_this_sample = {}
    imaging_request_this_sample = imaging_request_contents & \
        {'username':username,'request_name':request_name,'sample_name':sample_name}
    channel_contents_this_sample = channel_contents & \
        {'username':username,'request_name':request_name,'sample_name':sample_name}
    for channel_dict in channel_contents_this_sample:
        image_resolution = channel_dict['image_resolution']
        channel_name = channel_dict['channel_name']
        if image_resolution in imaging_dict_this_sample.keys():
            imaging_dict_this_sample[image_resolution].append(channel_name)
        else:
            imaging_dict_this_sample[image_resolution] = [channel_name]
    # Now check the imaging batch entries for a match to this imaging dict
    imaging_batch_contents_this_request =  db_lightsheet.Request.ImagingBatch() & \
        {'request_name':request_name}
    print(f"Sample name: {sample_name}")
    for imaging_batch_dict in imaging_batch_contents_this_request:
#         print("trying to match to imaging dict:")
#         print(imaging_batch_dict['imaging_dict'])
        if imaging_dict_this_sample == imaging_batch_dict['imaging_dict']:
            imaging_batch_number = imaging_batch_dict['imaging_batch_number']
            sample_reinsert_dict['imaging_batch_number'] = imaging_batch_number
            break
    else:
        print(f"No match to this imaging batch dict for sample {sample_name}:")
        print(imaging_dict_this_sample)
#     print()
    sample_reinsert_list.append(sample_reinsert_dict)

Sample name: sample-001
Sample name: sample-002
Sample name: sample-003
Sample name: sample-004
Sample name: sample-005
Sample name: sample-006
Sample name: sample-007
Sample name: sample-008
Sample name: sample-001
Sample name: sample-001
Sample name: zimmerman_01-001
Sample name: zimmerman_01-002
Sample name: mg80175-001
Sample name: sample-001
Sample name: sample-001
Sample name: sample-001
Sample name: sample-002
Sample name: sample-003
Sample name: sample-004
Sample name: sample-005
Sample name: sample-006
Sample name: sample-007
Sample name: sample-008
Sample name: sample-009
Sample name: sample-010
Sample name: sample-001
Sample name: sample-002
Sample name: sample-003
Sample name: sample-001
Sample name: sample-002
Sample name: sample-003
Sample name: sample-004
Sample name: sample-005
Sample name: sample-006
Sample name: sample-007
Sample name: sample-008
Sample name: three_female_atlas_brains-001
Sample name: three_female_atlas_brains-002
Sample name: three_female_atlas_brain

Sample name: ymazecfos_learning_verpeut-005
Sample name: ymazecfos_learning_verpeut-006
Sample name: ymazecfos_learning_verpeut-007
Sample name: ymazecfos_learning_verpeut-008
Sample name: ymazecfos_learning_verpeut-009
Sample name: ymazecfos_learning_verpeut-010
Sample name: ymazecfos_learning_verpeut-011
Sample name: ymazecfos_learning_verpeut-012
Sample name: ymazecfos_learning_verpeut-013
Sample name: ymazecfos_learning_verpeut-014
Sample name: ymazecfos_learning_verpeut-015
Sample name: ymazecfos_learning_verpeut-016
Sample name: ymazecfos_learning_verpeut-017
Sample name: ymazecfos_learning_verpeut-018
Sample name: ymazecfos_learning_verpeut-019
Sample name: ymazecfos_learning_verpeut-020
Sample name: ymazecfos_learning_verpeut-021
Sample name: ymazecfos_learning_verpeut-022
Sample name: ymazecfos_learning_verpeut-023
Sample name: ymazecfos_learning_verpeut-024
Sample name: ymazecfos_learning_verpeut-025
Sample name: ymazecfos_learning_verpeut-026
Sample name: ymazecfos_learning_

In [13]:
db_lightsheet.Request.SampleCopy()

username user in the lab,request_name,sample_name,clearing_protocol,antibody1,antibody2,clearing_batch_number,imaging_batch_number,subject_fullname


In [14]:
db_lightsheet.Request.SampleCopy().insert(sample_reinsert_list)

In [87]:
db_lightsheet.Request.SampleCopy()

username user in the lab,request_name,sample_name,clearing_protocol,antibody1,antibody2,clearing_batch_number,imaging_batch_number,subject_fullname
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",sample-001,iDISCO+_immuno,SySy Rb antiFos 1:1000,Donkey antiRabbit 488 1:500,1,1,
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",sample-002,iDISCO+_immuno,SySy Rb antiFos 1:1000,Donkey antiRabbit 488 1:500,1,1,
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",sample-003,iDISCO+_immuno,SySy Rb antiFos 1:1000,Donkey antiRabbit 488 1:500,1,1,
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",sample-004,iDISCO+_immuno,SySy Rb antiFos 1:1000,Donkey antiRabbit 488 1:500,1,1,
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",sample-005,iDISCO+_immuno,SySy Rb antiFos 1:1000,Donkey antiRabbit 488 1:500,1,1,
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",sample-006,iDISCO+_immuno,SySy Rb antiFos 1:1000,Donkey antiRabbit 488 1:500,1,1,
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",sample-007,iDISCO+_immuno,SySy Rb antiFos 1:1000,Donkey antiRabbit 488 1:500,1,1,
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",sample-008,iDISCO+_immuno,SySy Rb antiFos 1:1000,Donkey antiRabbit 488 1:500,1,1,
ahoag,test123,test123-001,iDISCO abbreviated clearing,,,1,1,
ahoag,test_3p6x,test_3p6x-001,iDISCO abbreviated clearing,,,1,1,


Finally, need to go back and update the imaging_progress column of the ImagingBatch table based on ImagingRequest table. 


In [15]:
batch_contents = db_lightsheet.Request.ImagingBatch()
batch_contents

/home/ahoag/anaconda3/envs/lightserv/lib/python3.7/site-packages/datajoint/connection.py:215: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


username user in the lab,request_name,imaging_batch_number,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,imaging_dict
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",1,None,8,2019-11-18,17:22:11,None,incomplete,=BLOB=
apv2,20190313_IBL_DiI_1,1,None,1,2019-03-13,1:43:03,None,incomplete,=BLOB=
apv2,ibl_witten_04,1,None,1,2019-08-12,15:07:23,None,incomplete,=BLOB=
cz15,zimmerman_01,1,None,2,2020-10-27,10:10:45,None,incomplete,=BLOB=
eguthman,mg80175,1,None,1,2020-09-29,13:08:31,None,incomplete,=BLOB=
ejdennis,"10-13_brains,_names_TBD",1,None,1,2019-08-06,17:12:15,None,incomplete,=BLOB=
ejdennis,201905_atlas00x_where_x=1:n,1,None,1,2019-05-21,19:12:45,None,incomplete,=BLOB=
ejdennis,20190606_atlas00x_where_x=11-20,1,None,10,2019-06-04,10:43:37,None,incomplete,=BLOB=
ejdennis,"E112,_E126,_E137",1,None,3,2020-03-11,12:03:39,None,incomplete,=BLOB=
ejdennis,"K310_(CM-diI),_K315_(CM-diI),_K320,_K321,_K323,_K327,_K333,_K334",1,None,8,2020-02-16,14:11:46,None,incomplete,=BLOB=


In [20]:
for batch_dict in batch_contents:
    # find samples in this request with this imaging_batch_number
    username = batch_dict['username']
    request_name = batch_dict['request_name']
    imaging_batch_number = batch_dict['imaging_batch_number']
    print(username,request_name,imaging_batch_number)
    current_imaging_progress = batch_dict['imaging_progress']
    this_batch_contents = batch_contents & \
        {'username':username,
         'request_name':request_name,'imaging_batch_number':imaging_batch_number}
    sample_contents_this_batch = sample_contents & \
        {'username':username,
         'request_name':request_name,'imaging_batch_number':imaging_batch_number}
    sample_names = tuple(sample_contents_this_batch.fetch('sample_name'))
    if len(sample_names) > 1:
        imaging_request_contents_this_batch = imaging_request_contents & \
            {'username':username,
             'request_name':request_name} & f'sample_name IN {sample_names}'
    elif len(sample_names) == 1:
        imaging_request_contents_this_batch = imaging_request_contents & \
        {'username':username,'request_name':request_name,'sample_name':sample_names[0]}
    imaging_progress_array = imaging_request_contents_this_batch.fetch('imaging_progress')
    imager_array = imaging_request_contents_this_batch.fetch('imager')
    if all([x=='complete' for x in imaging_progress_array]):
        imaging_progress = 'complete'
    elif any([x=='in progress' for x in imaging_progress_array]):
        imaging_progress = 'in progress'
    else:
        imaging_progress = 'incomplete'
    # Now update the ImagingBatch entry
    if imaging_progress != current_imaging_progress:
        dj.Table._update(this_batch_contents,'imaging_progress',imaging_progress)
    if imager_array!=[]:
        imager = imager_array[0]
        # Now update the ImagingBatch entry's imager
        current_imager = batch_dict['imager']

        if imager != current_imager:
            dj.Table._update(this_batch_contents,'imager',imager)

afalkner MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6 1
apv2 20190313_IBL_DiI_1 1


/home/ahoag/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/ahoag/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


apv2 ibl_witten_04 1
cz15 zimmerman_01 1


/home/ahoag/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/home/ahoag/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


ejdennis 10-13_brains,_names_TBD 1
ejdennis 201905_atlas00x_where_x=1:n 1


...

/home/ahoag/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/ahoag/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


zhihaoz 190912_zz34 1
zhihaoz 20190903_gp5p3_gcamp6f_20dg 1


/home/ahoag/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/home/ahoag/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


eguthman mg80175 1


/home/ahoag/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
